In [1]:
# coding:utf-8
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import ensemble, metrics
from datetime import datetime as dt
from sklearn.preprocessing import LabelEncoder


# pandas
import pandas as pd

# LightGBM
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 交差検証
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

In [2]:
purchase_df = pd.read_csv('../input/purchase_record.csv')
user_df = pd.read_csv('../input/user_info.csv')
test_df = pd.read_csv('../input/purchase_record_test.csv')

In [3]:
purchase_df.shape

(1803574, 14)

In [4]:
purchase_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1803574 entries, 0 to 1803573
Data columns (total 14 columns):
user_id        object
purchase_id    object
date           object
product_id     object
parts_1        object
parts_2        object
parts_3        object
parts_4        object
parts_5        object
parts_6        object
parts_7        object
parts_8        object
parts_9        object
purchase       int64
dtypes: int64(1), object(13)
memory usage: 192.6+ MB


In [5]:
purchase_df.head()

,user_id,purchase_id,date,product_id,parts_1,parts_2,parts_3,parts_4,parts_5,parts_6,parts_7,parts_8,parts_9,purchase
0,uid_100,pid_0000000000,2017-09-27,product 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,uid_100,pid_0000000001,2017-09-27,product 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,uid_100002,pid_0000000002,2017-11-23,product 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,uid_100002,pid_0000000003,2017-11-23,product 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,uid_100003,pid_0000000004,2018-02-07,product 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [6]:
purchase_df.fillna(0, inplace=True)

In [7]:
purchase_df.purchase.value_counts()

0    928044
1    875530
Name: purchase, dtype: int64

In [8]:
user_df.shape

(189114, 32)

In [9]:
user_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 189114 entries, 0 to 189113
Data columns (total 32 columns):
user_id         189114 non-null object
date            189114 non-null object
attribute_1     189114 non-null object
attribute_2     189114 non-null object
attribute_3     189114 non-null object
attribute_4     189114 non-null bool
attribute_5     189114 non-null bool
attribute_6     189114 non-null bool
attribute_7     189114 non-null bool
attribute_8     189114 non-null bool
attribute_9     189114 non-null bool
attribute_10    189114 non-null bool
attribute_11    189114 non-null bool
attribute_12    189114 non-null bool
attribute_13    189114 non-null bool
attribute_14    189114 non-null bool
attribute_15    189114 non-null bool
attribute_16    189114 non-null bool
attribute_17    189114 non-null bool
attribute_18    189114 non-null bool
attribute_19    189114 non-null bool
attribute_20    189114 non-null bool
attribute_21    189114 non-null bool
attribute_22    189114 non-n

In [10]:
user_df.head()

,user_id,date,attribute_1,attribute_2,attribute_3,attribute_4,attribute_5,attribute_6,attribute_7,attribute_8,...,attribute_21,attribute_22,attribute_23,attribute_24,attribute_25,attribute_26,attribute_27,attribute_28,attribute_29,attribute_30
0,uid_100,2016-06-29,id 5,id 5,id 5,True,False,False,True,True,...,True,False,True,True,False,False,True,True,True,False
1,uid_100002,2016-01-06,id 28,id 9,id 5,False,False,True,True,False,...,False,False,True,True,True,True,True,True,True,False
2,uid_100003,2017-06-10,id 4,id 8,id 5,True,True,True,True,True,...,True,False,False,True,True,True,True,False,True,True
3,uid_100004,2017-07-20,id 40,id 25,id 9,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
4,uid_100006,2017-07-27,id 40,id 25,id 9,False,False,False,False,False,...,False,False,False,True,False,False,False,True,True,False


In [11]:
train_df = pd.merge(purchase_df, user_df, how='left', on='user_id')

In [12]:
train_df.head()

,user_id,purchase_id,date_x,product_id,parts_1,parts_2,parts_3,parts_4,parts_5,parts_6,...,attribute_21,attribute_22,attribute_23,attribute_24,attribute_25,attribute_26,attribute_27,attribute_28,attribute_29,attribute_30
0,uid_100,pid_0000000000,2017-09-27,product 2,0,0,0,0,0,0,...,True,False,True,True,False,False,True,True,True,False
1,uid_100,pid_0000000001,2017-09-27,product 2,0,0,0,0,0,0,...,True,False,True,True,False,False,True,True,True,False
2,uid_100002,pid_0000000002,2017-11-23,product 2,0,0,0,0,0,0,...,False,False,True,True,True,True,True,True,True,False
3,uid_100002,pid_0000000003,2017-11-23,product 2,0,0,0,0,0,0,...,False,False,True,True,True,True,True,True,True,False
4,uid_100003,pid_0000000004,2018-02-07,product 2,0,0,0,0,0,0,...,True,False,False,True,True,True,True,False,True,True


In [13]:
# train_dfから日付を削除する
print(train_df.drop('date_x', axis=1).head())
train_df = train_df.drop('date_x', axis=1)
train_df = train_df.drop('date_y', axis=1)
train_df = train_df.drop('product_id', axis=1)
train_df = train_df.drop('user_id', axis=1)

      user_id     purchase_id product_id parts_1 parts_2 parts_3 parts_4  \
0     uid_100  pid_0000000000  product 2       0       0       0       0   
1     uid_100  pid_0000000001  product 2       0       0       0       0   
2  uid_100002  pid_0000000002  product 2       0       0       0       0   
3  uid_100002  pid_0000000003  product 2       0       0       0       0   
4  uid_100003  pid_0000000004  product 2       0       0       0       0   

  parts_5 parts_6 parts_7  ... attribute_21 attribute_22  attribute_23  \
0       0       0       0  ...         True        False          True   
1       0       0       0  ...         True        False          True   
2       0       0       0  ...        False        False          True   
3       0       0       0  ...        False        False          True   
4       0       0       0  ...         True        False         False   

  attribute_24 attribute_25 attribute_26 attribute_27  attribute_28  \
0         True        False

In [14]:
train_df.head()

,purchase_id,parts_1,parts_2,parts_3,parts_4,parts_5,parts_6,parts_7,parts_8,parts_9,...,attribute_21,attribute_22,attribute_23,attribute_24,attribute_25,attribute_26,attribute_27,attribute_28,attribute_29,attribute_30
0,pid_0000000000,0,0,0,0,0,0,0,0,0,...,True,False,True,True,False,False,True,True,True,False
1,pid_0000000001,0,0,0,0,0,0,0,0,0,...,True,False,True,True,False,False,True,True,True,False
2,pid_0000000002,0,0,0,0,0,0,0,0,0,...,False,False,True,True,True,True,True,True,True,False
3,pid_0000000003,0,0,0,0,0,0,0,0,0,...,False,False,True,True,True,True,True,True,True,False
4,pid_0000000004,0,0,0,0,0,0,0,0,0,...,True,False,False,True,True,True,True,False,True,True


In [15]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1803574 entries, 0 to 1803573
Data columns (total 41 columns):
purchase_id     object
parts_1         object
parts_2         object
parts_3         object
parts_4         object
parts_5         object
parts_6         object
parts_7         object
parts_8         object
parts_9         object
purchase        int64
attribute_1     object
attribute_2     object
attribute_3     object
attribute_4     bool
attribute_5     bool
attribute_6     bool
attribute_7     bool
attribute_8     bool
attribute_9     bool
attribute_10    bool
attribute_11    bool
attribute_12    bool
attribute_13    bool
attribute_14    bool
attribute_15    bool
attribute_16    bool
attribute_17    bool
attribute_18    bool
attribute_19    bool
attribute_20    bool
attribute_21    bool
attribute_22    bool
attribute_23    bool
attribute_24    bool
attribute_25    bool
attribute_26    bool
attribute_27    bool
attribute_28    bool
attribute_29    bool
attribute_30    bool

In [16]:
train_df.shape

(1803574, 41)

In [17]:
train_df.isnull().sum().sum()

0

In [18]:
# %%time
# pdp.ProfileReport(train_df)

In [19]:
test_df = pd.merge(test_df, user_df, how='left', on='user_id')

In [20]:
test_df.shape

(215041, 44)

In [21]:
# sample_train_df = train_df.sample(n=len(train_df)//500)

In [22]:
# sample_test_df = test_df.sample(n=len(test_df)//500)

In [23]:
# train_df = sample_train_df
# test_df = sample_test_df

In [24]:
# test_dfから日付を削除する
print(test_df.drop('date_x', axis=1).head())
test_df = test_df.drop('date_x', axis=1)
test_df = test_df.drop('date_y', axis=1)
test_df = test_df.drop('product_id', axis=1)
test_df = test_df.drop('user_id', axis=1)

      user_id     purchase_id product_id parts_1 parts_2 parts_3 parts_4  \
0     uid_100  pid_0001803575  product 4     NaN     NaN     NaN     NaN   
1     uid_100  pid_0001803576  product 2     NaN     NaN     NaN     NaN   
2     uid_100  pid_0001803577  product 2     NaN     NaN     NaN     NaN   
3     uid_100  pid_0001803578  product 4     NaN     NaN     NaN     NaN   
4  uid_100003  pid_0001803579  product 2     NaN     NaN     NaN     NaN   

  parts_5 parts_6 parts_7  ... attribute_21 attribute_22 attribute_23  \
0     NaN     NaN     NaN  ...         True        False         True   
1     NaN     NaN     NaN  ...         True        False         True   
2     NaN     NaN     NaN  ...         True        False         True   
3     NaN     NaN     NaN  ...         True        False         True   
4     NaN     NaN     NaN  ...         True        False        False   

  attribute_24 attribute_25 attribute_26  attribute_27  attribute_28  \
0         True        False     

In [25]:
test_df.head()

,purchase_id,parts_1,parts_2,parts_3,parts_4,parts_5,parts_6,parts_7,parts_8,parts_9,...,attribute_21,attribute_22,attribute_23,attribute_24,attribute_25,attribute_26,attribute_27,attribute_28,attribute_29,attribute_30
0,pid_0001803575,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,True,False,True,True,False,False,True,True,True,False
1,pid_0001803576,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,True,False,True,True,False,False,True,True,True,False
2,pid_0001803577,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,True,False,True,True,False,False,True,True,True,False
3,pid_0001803578,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,True,False,True,True,False,False,True,True,True,False
4,pid_0001803579,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,True,False,False,True,True,True,True,False,True,True


In [26]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1803574 entries, 0 to 1803573
Data columns (total 41 columns):
purchase_id     object
parts_1         object
parts_2         object
parts_3         object
parts_4         object
parts_5         object
parts_6         object
parts_7         object
parts_8         object
parts_9         object
purchase        int64
attribute_1     object
attribute_2     object
attribute_3     object
attribute_4     bool
attribute_5     bool
attribute_6     bool
attribute_7     bool
attribute_8     bool
attribute_9     bool
attribute_10    bool
attribute_11    bool
attribute_12    bool
attribute_13    bool
attribute_14    bool
attribute_15    bool
attribute_16    bool
attribute_17    bool
attribute_18    bool
attribute_19    bool
attribute_20    bool
attribute_21    bool
attribute_22    bool
attribute_23    bool
attribute_24    bool
attribute_25    bool
attribute_26    bool
attribute_27    bool
attribute_28    bool
attribute_29    bool
attribute_30    bool

In [27]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 215041 entries, 0 to 215040
Data columns (total 40 columns):
purchase_id     215041 non-null object
parts_1         21006 non-null object
parts_2         21006 non-null object
parts_3         21006 non-null object
parts_4         21006 non-null object
parts_5         21006 non-null object
parts_6         21006 non-null object
parts_7         21006 non-null object
parts_8         21006 non-null object
parts_9         21006 non-null object
attribute_1     215041 non-null object
attribute_2     215041 non-null object
attribute_3     215041 non-null object
attribute_4     215041 non-null bool
attribute_5     215041 non-null bool
attribute_6     215041 non-null bool
attribute_7     215041 non-null bool
attribute_8     215041 non-null bool
attribute_9     215041 non-null bool
attribute_10    215041 non-null bool
attribute_11    215041 non-null bool
attribute_12    215041 non-null bool
attribute_13    215041 non-null bool
attribute_14    21504

In [28]:
%%time
# train = pd.get_dummies(train_df, drop_first=True)
train = pd.get_dummies(train_df, columns=['attribute_1', 'attribute_2', 'parts_1', 'parts_2', 'parts_3', 'parts_4', 'parts_5', 'parts_6', 'parts_7', 'parts_8', 'parts_9'], drop_first=True)
train * 1
# train = pd.get_dummies(train_df, columns!=['purchase_id', 'purchase'], drop_first=True)



CPU times: user 3.21 s, sys: 1.02 s, total: 4.23 s
Wall time: 4.24 s


,purchase_id,purchase,attribute_3,attribute_4,attribute_5,attribute_6,attribute_7,attribute_8,attribute_9,attribute_10,...,parts_9_id 18,parts_9_id 19,parts_9_id 2,parts_9_id 3,parts_9_id 4,parts_9_id 5,parts_9_id 6,parts_9_id 7,parts_9_id 8,parts_9_id 9
0,pid_0000000000,0,id 5,1,0,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
1,pid_0000000001,0,id 5,1,0,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
2,pid_0000000002,1,id 5,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,pid_0000000003,1,id 5,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,pid_0000000004,1,id 5,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1803569,pid_0001803569,1,id 2,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1803570,pid_0001803570,1,id 2,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1803571,pid_0001803571,1,id 2,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1803572,pid_0001803572,1,id 2,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
%%time
# test = pd.get_dummies(test_df, drop_first=True)
#test = pd.get_dummies(test_df, drop_first=True)
test = pd.get_dummies(test_df, columns=['attribute_1', 'attribute_2', 'parts_1', 'parts_2', 'parts_3', 'parts_4', 'parts_5', 'parts_6', 'parts_7', 'parts_8', 'parts_9'], drop_first=True)
test * 1

In [ ]:
train.info()

In [ ]:
test_df.info()

In [ ]:
# data_yに目的変数を代入
y_train = train['purchase']

# data_yの表示
print( y_train )

# data_Xに説明変数を代入
X_train = train.drop('purchase', axis=1)

# data_Xの表示
print( X_train )

In [ ]:
%%time
# 10分割交差検証を指定し、インスタンス化
n_splits = 5
kf = KFold(n_splits, shuffle=True)

# スコアとモデルを格納するリスト
score_list = []
models = []

for fold_, (train_index, valid_index) in enumerate(kf.split(X_train, y_train)):
    print(f'fold{fold_ + 1} start')
    train_x = X_train.iloc[train_index]
    valid_x = X_train.iloc[valid_index]
    train_y = y_train[train_index]
    valid_y = y_train[valid_index]
    # lab.Datasetを使って,trainとvalidを作っておく
    lgb_train= lgb.Dataset(train_x, train_y)
    lgb_valid = lgb.Dataset(valid_x, valid_y)

    # パラメータを定義
    lgbm_params = {'objective': 'binary'}                                                                                               
          
    # 学習
    gbm = lgb.train(params = lgbm_params,
                    train_set = lgb_train, 
                    num_boost_round = 200,
                    valid_sets = [lgb_train, lgb_valid],
                    early_stopping_rounds = 20,
                    verbose_eval = -1 # 学習の状況を表示しない
                   )

    oof = gbm.predict(valid_x)
    # print(oof)
    score_list.append(np.sqrt(mean_squared_error(valid_y, oof))) #RMSEを出す
    # score_list.append(round(accuracy_score(valid_y, oof)*100,2)) # 正解率Accuracyを出す。(検証用y, 予測結果)
    print('RMSE:', score_list)
    models.append(gbm) # 学習が終わったモデルをリストに入れておく
    print(f'fold{fold_ + 1} end\n' )
print(score_list, '平均score', round(np.mean(score_list), 2))

In [ ]:
# 比較
p = pd.DataFrame({"actual":valid_y,"pred":oof})
p.plot(figsize=(15,4))
print("RMSE",round(np.mean(score_list), 2))

In [ ]:
# テストデータの予測を格納する。テストデータ列数*分割数列のnumpy行列を作成
test_pred = np.zeros((len(df_test), n_splits))

for fold_, gbm in enumerate(models):
    pred_ = gbm.predict(df_test) # testを予測
    test_pred[:, fold_] = pred_

result = pd.DataFrame({
    'purchase_id': df_test.index.values,
    'probability': np.mean(test_pred, axis=1)
})
result.to_csv('../output/submit.csv',header=False,index=False)

In [ ]:
print("予測完了")